# LAB Monday: Scrape the current top 100 songs and their respective artists, and put the information into a pandas dataframe.

In [2]:
#Importing required libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd

#Define the url to scrape the Hot 100 songs data
url = 'https://www.billboard.com/charts/hot-100/'

#Make a GET request to the URL
results = requests.get(url, headers = {'Accept-Language':'en-US'})

#Create a BeautifulSoup object from the content of the request
soup = BeautifulSoup(results.content, 'html.parser')

#Scrape the song title and artist name from the HTML
song_titles = [song.get_text(strip=True) for song in soup.select('h3.c-title.a-no-trucate')]
artists = [artist.get_text(strip=True) for artist in soup.select('span.c-label.a-no-trucate')]

#Create a dataframe to store the scraped data
songs = pd.DataFrame({'Song Title': song_titles, 'Artist': artists})

#Display the dataframe

In [3]:
songs 

,Song Title,Artist
0,Flowers,Miley Cyrus
1,Kill Bill,SZA
2,Anti-Hero,Taylor Swift
3,Creepin',"Metro Boomin, The Weeknd & 21 Savage"
4,Unholy,Sam Smith & Kim Petras
...,...,...
95,Do It Again,NLE Choppa & 2Rare
96,Miss You,Oliver Tree & Robin Schulz
97,Back End,Finesse2Tymes
98,Open Arms,SZA Featuring Travis Scott


# LAB Wednesday: API wrappers - Create your collection of songs & audio features

In [4]:
# Import the required libraries
import spotipy 
from spotipy.oauth2 import SpotifyClientCredentials
import getpass

# Request the user's Spotify client ID and secret
client_id = str(getpass.getpass('client_id?'))
client_secret = str(getpass.getpass('client_secret?'))

# Create a client credentials manager with the client ID and secret
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

# Get the tracks from the specified playlist using the user's Spotify account
playlist = sp.user_playlist_tracks('Bradnumber1','1YL4XoegERoragv0RK2RC9')

# Print all the keys of the playlist dictionary
playlist.keys()

# Get the ID of the 5th track in the playlist
playlist['items'][4]['track']['id']

# Get the total number of tracks in the playlist
playlist['total']

# Define a function to get all the tracks from the specified playlist
def get_playlist_tracks(user_id,playlist_id):
    # Get the first set of tracks from the playlist
    results = sp.user_playlist_tracks(user_id,playlist_id)
    tracks = results['items']
    # Continue getting tracks while there are more
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

# Get all the tracks from the specified playlist
full_track_ids = get_playlist_tracks('Bradnumber1','1YL4XoegERoragv0RK2RC9')

# Initialize lists to store the title, artist, and ID of each track
title = []
artist = []
song_id = []

# Loop through each track in the full list of track IDs
for i in full_track_ids:
    # Add the title, artist, and ID of each track to the respective lists
    title.append(i['track']['name'])
    artist.append(i['track']['artists'][0]['name'])
    song_id.append(i['track']['id'])

# Create a Pandas dataframe from the title, artist, and ID lists
playlist_df = pd.DataFrame({'title': title, 'artist': artist, 'song_id': song_id})


client_id? ········
client_secret? ········


In [6]:
playlist_df
#ab2bba45532140f88416bc2ec8147634
#211681fd96344fe7bf32cc288d007866

,title,artist,song_id
0,Suspicious Minds,Elvis Presley,1H5IfYyIIAlgDX8zguUzns
1,And We Danced,The Hooters,5xkNd7kCRmjT2DxSAYVVyC
2,Love Theme from St. Elmo's Fire - Instrumental,David Foster,65Wspymr1UkFIT0WRyVJnI
3,Better - Remastered,The Screaming Jets,6VtSR2Nw3IozhsRmsKzIET
4,Accidentally In Love,Counting Crows,4FXdIM78OBdw7KIY2jeM8D
...,...,...,...
9982,Growing on Me,The Darkness,77asYwewm0lXvz76inosJm
9983,One Way Ticket,The Darkness,44x9GdJsYmbaYRBm8ROyfL
9984,Givin' Up,The Darkness,3LDsZ7JCzTwfuTRuteUqtB
9985,Gypsy Woman (She's Homeless) (La Da Dee La Da ...,Crystal Waters,4QAKfScH8kLJTbJqhb2jp2


In [10]:
#Create an empty list to store audio features for each track
spaceplaylist = []

#Loop through the songs in the playlist and retrieve audio features for each track
for i in range(len(playlist_df)):
    try :
        spaceplaylist.extend(sp.audio_features(tracks=playlist_df['song_id'][i]))
    except :
        print(i)

1925
9270


In [11]:
#Create a dataframe from the audio features data
song_features = pd.DataFrame.from_dict(spaceplaylist)

In [12]:
song_features

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.487,0.382,7,-10.889,1,0.0309,0.042200,0.000005,0.4110,0.714,116.557,audio_features,1H5IfYyIIAlgDX8zguUzns,spotify:track:1H5IfYyIIAlgDX8zguUzns,https://api.spotify.com/v1/tracks/1H5IfYyIIAlg...,https://api.spotify.com/v1/audio-analysis/1H5I...,261280,4
1,0.561,0.945,9,-6.086,1,0.0444,0.086900,0.000000,0.2200,0.723,145.781,audio_features,5xkNd7kCRmjT2DxSAYVVyC,spotify:track:5xkNd7kCRmjT2DxSAYVVyC,https://api.spotify.com/v1/tracks/5xkNd7kCRmjT...,https://api.spotify.com/v1/audio-analysis/5xkN...,227760,4
2,0.570,0.404,8,-13.761,1,0.0361,0.501000,0.668000,0.1040,0.312,125.201,audio_features,65Wspymr1UkFIT0WRyVJnI,spotify:track:65Wspymr1UkFIT0WRyVJnI,https://api.spotify.com/v1/tracks/65Wspymr1UkF...,https://api.spotify.com/v1/audio-analysis/65Ws...,209400,4
3,0.486,0.819,2,-5.334,1,0.0711,0.000226,0.000000,0.0366,0.439,101.536,audio_features,6VtSR2Nw3IozhsRmsKzIET,spotify:track:6VtSR2Nw3IozhsRmsKzIET,https://api.spotify.com/v1/tracks/6VtSR2Nw3Ioz...,https://api.spotify.com/v1/audio-analysis/6VtS...,277680,4
4,0.555,0.926,7,-3.613,1,0.0363,0.048400,0.000008,0.1710,0.771,138.017,audio_features,4FXdIM78OBdw7KIY2jeM8D,spotify:track:4FXdIM78OBdw7KIY2jeM8D,https://api.spotify.com/v1/tracks/4FXdIM78OBdw...,https://api.spotify.com/v1/audio-analysis/4FXd...,188387,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9980,0.401,0.848,2,-4.247,1,0.0484,0.000340,0.000023,0.3420,0.487,136.214,audio_features,77asYwewm0lXvz76inosJm,spotify:track:77asYwewm0lXvz76inosJm,https://api.spotify.com/v1/tracks/77asYwewm0lX...,https://api.spotify.com/v1/audio-analysis/77as...,208880,4
9981,0.506,0.761,11,-5.603,1,0.0973,0.012200,0.000007,0.0860,0.491,132.436,audio_features,44x9GdJsYmbaYRBm8ROyfL,spotify:track:44x9GdJsYmbaYRBm8ROyfL,https://api.spotify.com/v1/tracks/44x9GdJsYmba...,https://api.spotify.com/v1/audio-analysis/44x9...,266973,4
9982,0.560,0.927,11,-3.126,1,0.0503,0.016200,0.000165,0.2360,0.818,136.613,audio_features,3LDsZ7JCzTwfuTRuteUqtB,spotify:track:3LDsZ7JCzTwfuTRuteUqtB,https://api.spotify.com/v1/tracks/3LDsZ7JCzTwf...,https://api.spotify.com/v1/audio-analysis/3LDs...,214960,4
9983,0.645,0.779,9,-10.529,0,0.0315,0.000165,0.214000,0.0941,0.453,120.007,audio_features,4QAKfScH8kLJTbJqhb2jp2,spotify:track:4QAKfScH8kLJTbJqhb2jp2,https://api.spotify.com/v1/tracks/4QAKfScH8kLJ...,https://api.spotify.com/v1/audio-analysis/4QAK...,229573,4


In [13]:
#Merge the playlist data and audio features data into one dataframe
playlist_features = playlist_df.merge(song_features, left_on='song_id', right_on='id')

In [14]:
playlist_features

,title,artist,song_id,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Suspicious Minds,Elvis Presley,1H5IfYyIIAlgDX8zguUzns,0.487,0.382,7,-10.889,1,0.0309,0.042200,...,0.4110,0.714,116.557,audio_features,1H5IfYyIIAlgDX8zguUzns,spotify:track:1H5IfYyIIAlgDX8zguUzns,https://api.spotify.com/v1/tracks/1H5IfYyIIAlg...,https://api.spotify.com/v1/audio-analysis/1H5I...,261280,4
1,And We Danced,The Hooters,5xkNd7kCRmjT2DxSAYVVyC,0.561,0.945,9,-6.086,1,0.0444,0.086900,...,0.2200,0.723,145.781,audio_features,5xkNd7kCRmjT2DxSAYVVyC,spotify:track:5xkNd7kCRmjT2DxSAYVVyC,https://api.spotify.com/v1/tracks/5xkNd7kCRmjT...,https://api.spotify.com/v1/audio-analysis/5xkN...,227760,4
2,Love Theme from St. Elmo's Fire - Instrumental,David Foster,65Wspymr1UkFIT0WRyVJnI,0.570,0.404,8,-13.761,1,0.0361,0.501000,...,0.1040,0.312,125.201,audio_features,65Wspymr1UkFIT0WRyVJnI,spotify:track:65Wspymr1UkFIT0WRyVJnI,https://api.spotify.com/v1/tracks/65Wspymr1UkF...,https://api.spotify.com/v1/audio-analysis/65Ws...,209400,4
3,Better - Remastered,The Screaming Jets,6VtSR2Nw3IozhsRmsKzIET,0.486,0.819,2,-5.334,1,0.0711,0.000226,...,0.0366,0.439,101.536,audio_features,6VtSR2Nw3IozhsRmsKzIET,spotify:track:6VtSR2Nw3IozhsRmsKzIET,https://api.spotify.com/v1/tracks/6VtSR2Nw3Ioz...,https://api.spotify.com/v1/audio-analysis/6VtS...,277680,4
4,Accidentally In Love,Counting Crows,4FXdIM78OBdw7KIY2jeM8D,0.555,0.926,7,-3.613,1,0.0363,0.048400,...,0.1710,0.771,138.017,audio_features,4FXdIM78OBdw7KIY2jeM8D,spotify:track:4FXdIM78OBdw7KIY2jeM8D,https://api.spotify.com/v1/tracks/4FXdIM78OBdw...,https://api.spotify.com/v1/audio-analysis/4FXd...,188387,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10080,Growing on Me,The Darkness,77asYwewm0lXvz76inosJm,0.401,0.848,2,-4.247,1,0.0484,0.000340,...,0.3420,0.487,136.214,audio_features,77asYwewm0lXvz76inosJm,spotify:track:77asYwewm0lXvz76inosJm,https://api.spotify.com/v1/tracks/77asYwewm0lX...,https://api.spotify.com/v1/audio-analysis/77as...,208880,4
10081,One Way Ticket,The Darkness,44x9GdJsYmbaYRBm8ROyfL,0.506,0.761,11,-5.603,1,0.0973,0.012200,...,0.0860,0.491,132.436,audio_features,44x9GdJsYmbaYRBm8ROyfL,spotify:track:44x9GdJsYmbaYRBm8ROyfL,https://api.spotify.com/v1/tracks/44x9GdJsYmba...,https://api.spotify.com/v1/audio-analysis/44x9...,266973,4
10082,Givin' Up,The Darkness,3LDsZ7JCzTwfuTRuteUqtB,0.560,0.927,11,-3.126,1,0.0503,0.016200,...,0.2360,0.818,136.613,audio_features,3LDsZ7JCzTwfuTRuteUqtB,spotify:track:3LDsZ7JCzTwfuTRuteUqtB,https://api.spotify.com/v1/tracks/3LDsZ7JCzTwf...,https://api.spotify.com/v1/audio-analysis/3LDs...,214960,4
10083,Gypsy Woman (She's Homeless) (La Da Dee La Da ...,Crystal Waters,4QAKfScH8kLJTbJqhb2jp2,0.645,0.779,9,-10.529,0,0.0315,0.000165,...,0.0941,0.453,120.007,audio_features,4QAKfScH8kLJTbJqhb2jp2,spotify:track:4QAKfScH8kLJTbJqhb2jp2,https://api.spotify.com/v1/tracks/4QAKfScH8kLJ...,https://api.spotify.com/v1/audio-analysis/4QAK...,229573,4
